# 이미지 이해

https://ai.google.dev/gemini-api/docs/image-understanding?hl=ko

In [1]:
!pip install -q google-genai python-dotenv

In [2]:
!pip install -q pillow
from PIL import Image

In [3]:
# 키 로딩
import os, dotenv
dotenv.load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
# GEMINI_API_KEY # 이 출력 결과도 커밋이 되지 않도록 해야 함.

## 이미지 입력

다음과 같은 방법으로 Gemini에 이미지를 입력으로 제공할 수 있습니다.

generateContent 에 요청하기 전에, File API 를 사용하여 이미지 파일을 업로드하세요. 20MB가 넘는 파일이나 여러 요청에 걸쳐 파일을 재사용하려는 경우 이 방법을 사용하세요.
요청과 함께 인라인 이미지 데이터를 전달합니다generateContent . 이 방법은 작은 파일(총 요청 크기가 20MB 미만)이나 URL에서 직접 가져온 이미지에 사용하세요.


### 이미지 파일 업로드

- Files API를 사용하여 이미지 파일을 업로드할 수 있습니다. 
- 총 요청 크기 (파일, 텍스트 프롬프트, 시스템 안내 등 포함)가 "20MB를 초과"하거나, 여러 프롬프트에서 동일한 이미지를 사용할 계획인 경우 항상 Files API를 사용하세요.

In [ ]:
from google import genai
client = genai.Client(api_key=GEMINI_API_KEY)

my_file = client.files.upload(file="cat.jpg")
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[my_file, "Caption this image."],
)
# ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}

print(response.text)

### 인라인으로 이미지 데이터 전달

- 이미지 파일을 업로드하는 대신 요청에서 인라인 이미지 데이터를 generateContent에 전달할 수 있습니다. 이는 작은 이미지(총 요청 크기가 20MB 미만) 또는 URL에서 직접 가져온 이미지에 적합합니다.
- 이미지 데이터를 Base64로 인코딩된 문자열로 제공하거나 로컬 파일을 직접 읽어서 제공할 수 있습니다 (SDK에 따라 다름).

In [ ]:
# 로컬 이미지 파일
from google.genai import types

with open('path/to/small-sample.jpg', 'rb') as f:
    image_bytes = f.read()

response = client.models.generate_content(
model='gemini-2.0-flash',
contents=[
    types.Part.from_bytes(
    data=image_bytes,
    mime_type='image/jpeg',
    ),
    'Caption this image.'
]
)

print(response.text)

In [ ]:
# URL 이미지
from google import genai
from google.genai import types

import requests

image_path = "https://goo.gle/instrument-img"
image_bytes = requests.get(image_path).content
image = types.Part.from_bytes(
  data=image_bytes, mime_type="image/jpeg"
)

client = genai.Client(api_key="GOOGLE_API_KEY")
response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents=["What is this image?", image],
)
print(response.text)

## 여러 이미지를 사용한 프롬프트

contents 배열에 여러 이미지 Part 객체를 포함하여 단일 프롬프트에 여러 이미지를 제공할 수 있습니다. 인라인 데이터(로컬 파일 또는 URL)와 File API 참조가 혼합될 수 있습니다.

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key="GOOGLE_API_KEY")

# Upload the first image
image1_path = "path/to/image1.jpg"
uploaded_file = client.files.upload(file=image1_path)

# Prepare the second image as inline data
image2_path = "path/to/image2.png"
with open(image2_path, 'rb') as f:
    img2_bytes = f.read()

# Create the prompt with text and multiple images
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[
        "What is different between these two images?",
        uploaded_file,  # Use the uploaded file reference
        types.Part.from_bytes(
            data=img2_bytes,
            mime_type='image/png'
        )
    ]
)
print(response.text)

## 객체의 경계 상자 가져오기

Gemini 모델은 이미지에서 객체를 식별하고 경계 상자 좌표를 제공하도록 학습됩니다. 좌표는 이미지 크기를 기준으로 반환되며 [0, 1000]으로 크기가 조정됩니다. 원본 이미지 크기에 따라 이러한 좌표의 크기를 조정해야 합니다.

In [ ]:
prompt = "Detect the all of the prominent items in the image. The box_2d should be [ymin, xmin, ymax, xmax] normalized to 0-1000."

## 이미지 분할 (segmentation)

Gemini 2.5 모델부터, Gemini 모델은 항목을 감지할 뿐만 아니라 항목을 분할하고 윤곽의 마스크를 제공하도록 학습됩니다.

모델은 각 항목이 세분화 마스크를 나타내는 JSON 목록을 예측합니다. 각 항목에는 0~1000 사이의 정규화된 좌표를 갖는 [y0, x0, y1, x1] 형식의 경계 상자('box_2d'), 객체를 식별하는 라벨('label'), 마지막으로 경계 상자 내부의 분할 마스크(값이 0~255 사이인 확률 맵인 base64 인코딩된 png)가 있습니다. 마스크의 크기를 경계 상자 크기와 일치하도록 조정한 다음 신뢰도 임계값 (중점의 경우 127)에서 바이너리화해야 합니다.

In [ ]:
prompt = """
  Give the segmentation masks for the wooden and glass items.
  Output a JSON list of segmentation masks where each entry contains the 2D
  bounding box in the key "box_2d", the segmentation mask in key "mask", and
  the text label in the key "label". Use descriptive labels.
"""

## 이미지에 관한 기술 세부정보
- 파일 한도: Gemini 2.5 Pro, 2.0 Flash, 1.5 Pro, 1.5 Flash는 요청당 최대 3,600개의 이미지 파일을 지원합니다.
- 토큰 계산:
  - Gemini 1.5 Flash 및 Gemini 1.5 Pro:
    - 두 크기가 모두 384픽셀 이하인 경우 258개 토큰
    - 더 큰 이미지는 타일로 표시됨 (최소 타일 256px, 최대 768px, 768x768로 크기 조정). 각 타일은 258개의 토큰이 소요.
  - Gemini 2.0 Flash:
    - 두 측정기준이 모두 384픽셀 이하인 경우 258개 토큰 더 큰 이미지는 768x768픽셀 타일로 타일링되며, 각각 258개의 토큰이 소요.
- 권장사항:
  - 이미지가 올바르게 회전되었는지 확인합니다.
  - 선명하고 흐릿하지 않은 이미지를 사용하세요.
  - 텍스트가 포함된 단일 이미지를 사용하는 경우 contents 배열의 이미지 부분 뒤에 텍스트 프롬프트를 배치합니다.

끝